In [1]:
import numpy as np

import torch
from torch.autograd import grad

In [2]:
if torch.cuda.is_available():
    _DEVICE = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    _DEVICE = torch.device('cpu')
    torch.set_default_tensor_type('torch.FloatTensor')

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Use `autograd.grad` to get $2^{\rm nd}$ derivative
Based on https://discuss.pytorch.org/t/calculating-second-derivative/97989

In [3]:
def second_derivative(y, x):

    ncols = x.shape[1]
    dy_dx = torch.zeros(x.shape, device=_DEVICE)
    d2y_dx2 = torch.zeros(x.shape, device=_DEVICE)
    for i in range(ncols):
        g = grad(y[:, i].unsqueeze(1), x,
                 torch.ones(x.size()[0], 1, device=_DEVICE),
                 create_graph=True, retain_graph=True)[0]
        gg = grad(g[:,i].unsqueeze(1), x,
                  torch.ones(x.size()[0], 1, device=_DEVICE),
                  create_graph=True, retain_graph=True)[0]
        dy_dx += g
        d2y_dx2 += gg
    print('1st derivative', dy_dx, 3*x**2+2, sep='\n')
    print('2nd derivative', d2y_dx2, 6*x, sep='\n')
            
    return d2y_dx2

x = 2*torch.rand((5,3))
x.requires_grad_(True)
x.retain_grad()
print(x)
y = x**3 + 2*x
print(y)
sec_der = second_derivative(y,x)

tensor([[1.5617, 0.1999, 0.8343],
        [0.3741, 0.8439, 0.3889],
        [0.8514, 1.3458, 0.8169],
        [0.4942, 1.9762, 0.3306],
        [0.5176, 1.3946, 0.3305]], requires_grad=True)
tensor([[ 6.9322,  0.4079,  2.2491],
        [ 0.8006,  2.2887,  0.8365],
        [ 2.3200,  5.1290,  2.1790],
        [ 1.1091, 11.6706,  0.6974],
        [ 1.1740,  5.5017,  0.6971]], grad_fn=<AddBackward0>)
1st derivative
tensor([[ 9.3166,  2.1199,  4.0879],
        [ 2.4198,  4.1364,  2.4537],
        [ 4.1747,  7.4335,  4.0020],
        [ 2.7327, 13.7164,  2.3279],
        [ 2.8039,  7.8348,  2.3277]], grad_fn=<AddBackward0>)
tensor([[ 9.3166,  2.1199,  4.0879],
        [ 2.4198,  4.1364,  2.4537],
        [ 4.1747,  7.4335,  4.0020],
        [ 2.7327, 13.7164,  2.3279],
        [ 2.8039,  7.8348,  2.3277]], grad_fn=<AddBackward0>)
2nd derivative
tensor([[ 9.3702,  1.1997,  5.0055],
        [ 2.2446,  5.0632,  2.3332],
        [ 5.1085,  8.0748,  4.9014],
        [ 2.9652, 11.8574,  1.9837],
 

## Use `autograd.grad` to convert optimisation problem to a linear system of equations

In [4]:
x = torch.arange(10.)
y = 2*x + 3
print(x, y, sep='\n')

a = torch.zeros((), device=_DEVICE, requires_grad=True)
b = torch.zeros((), device=_DEVICE, requires_grad=True)

loss = .5*(a*x+b-y).pow(2).sum()
print(loss)

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
tensor([ 3.,  5.,  7.,  9., 11., 13., 15., 17., 19., 21.])
tensor(885., grad_fn=<MulBackward0>)


In [5]:
def solve(A, b):
    A_ = A.detach().numpy()
    b_ = b.detach().numpy()
    soln = np.linalg.solve(A_, b_)
    print(soln)

In [6]:
mat = torch.FloatTensor([[(x**2).sum(), x.sum()], [x.sum(), 10.]])
vec = torch.FloatTensor([(x*y).sum(), y.sum()])
print(mat, vec, sep='\n')
solve(mat, vec)

tensor([[285.,  45.],
        [ 45.,  10.]])
tensor([705., 120.])
[2. 3.]


In [7]:
d_a = grad(loss, a, create_graph=True, retain_graph=True  )[0]
d_b = grad(loss, b, create_graph=True, retain_graph=True  )[0]
vec_ = - torch.FloatTensor([d_a, d_b])
print(vec_)

tensor([705., 120.])


In [8]:
d_aa = grad(d_a, a, create_graph=True, retain_graph=True  )[0]
d_ab = grad(d_a, b, create_graph=True, retain_graph=True  )[0]
d_ba = grad(d_b, a, create_graph=True, retain_graph=True  )[0]
d_bb = grad(d_b, b, create_graph=True, retain_graph=True  )[0]
mat_ = torch.FloatTensor([[d_aa, d_ab], [d_ba, d_bb]])
print(mat_)
solve(mat_, vec_)

tensor([[285.,  45.],
        [ 45.,  10.]])
[2. 3.]


## Use `autograd.grad` to convert optimisation problem to a linear system of equations
Use vector operations this time

In [9]:
x = torch.arange(10.)
y = 2*x + 3
ab = torch.zeros((2,), device=_DEVICE, requires_grad=True)
loss = .5*(ab[0]*x+ab[1]-y).pow(2).sum()

In [10]:
g = grad(loss, ab, create_graph=True, retain_graph=True  )[0]
gg = torch.zeros((2,2), device=_DEVICE)
for i in range(2):
    gg[i,:] = grad(g[i], ab, create_graph=True, retain_graph=True  )[0]
solve(gg, -g)

[2. 3.]
